In [123]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [124]:
#convert images into pandas dataframes
from pathlib import Path
# path after mount: drive/MyDrive/archive (2)/chest_xray/train
#this is just us getting the data from google drive

local_dir = Path.cwd()
train_dir = local_dir / 'drive/MyDrive/archive (2)/chest_xray/train'
test_dir = local_dir / 'drive/MyDrive/archive (2)/chest_xray/test'
val_dir = local_dir / 'drive/MyDrive/archive (2)/chest_xray/val'


train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123, shuffle=True)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir, seed=123)


Found 2682 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [125]:
#below are the preprocessing layers

# Normalizes the data
rescale = tf.keras.Sequential([
  layers.Rescaling(1./255.0)
])

In [126]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [127]:
num_classes = 2
model = tf.keras.models.Sequential()
model.add(rescale) #normalizing
model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3)) #dropout added
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3)) #dropout added
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [132]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

history = model.fit(  train_ds,
  validation_data=val_ds,
  batch_size = 128,
  epochs=20)

Epoch 1/20
84/84 [==============================] - 6s 59ms/step - loss: 0.0988 - accuracy: 0.9609 - val_loss: 0.1788 - val_accuracy: 0.8750
Epoch 2/20
84/84 [==============================] - 5s 57ms/step - loss: 0.1257 - accuracy: 0.9635 - val_loss: 0.0185 - val_accuracy: 1.0000
Epoch 3/20
84/84 [==============================] - 5s 57ms/step - loss: 0.1380 - accuracy: 0.9534 - val_loss: 0.0531 - val_accuracy: 1.0000
Epoch 4/20
84/84 [==============================] - 5s 57ms/step - loss: 0.1157 - accuracy: 0.9631 - val_loss: 0.0716 - val_accuracy: 0.9375
Epoch 5/20
84/84 [==============================] - 5s 57ms/step - loss: 0.0898 - accuracy: 0.9672 - val_loss: 2.0594 - val_accuracy: 0.8125
Epoch 6/20
84/84 [==============================] - 5s 57ms/step - loss: 0.3785 - accuracy: 0.9612 - val_loss: 0.6650 - val_accuracy: 0.9375
Epoch 7/20
84/84 [==============================] - 5s 57ms/step - loss: 0.4706 - accuracy: 0.8725 - val_loss: 0.0533 - val_accuracy: 1.0000
Epoch 8/20
84

In [134]:
score = model.evaluate(test_ds)

20/20 [==============================] - 2s 67ms/step - loss: 1.2519 - accuracy: 0.7997
